---
## Setup Security

In [ ]:
-- USE ROLE SECURITYADMIN;

-- GRANT USAGE ON DATABASE HISTORICAL_QUOTES_DEMO TO ROLE OPENFLOW_RUNTIME1_ROLE;
-- GRANT USAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE OPENFLOW_RUNTIME1_ROLE;
-- GRANT CREATE TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE OPENFLOW_RUNTIME1_ROLE;

-- GRANT USAGE ON DATABASE HISTORICAL_QUOTES_DEMO TO ROLE ACCOUNTADMIN;
-- GRANT USAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;
-- GRANT SELECT ON ALL TABLES IN SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC to ROLE ACCOUNTADMIN;
-- GRANT CREATE TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;
-- GRANT CREATE DYNAMIC TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;

-- GRANT CREATE STREAMLIT ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE PUBLIC;
-- GRANT CREATE STAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE PUBLIC;
-- GRANT USAGE ON WAREHOUSE "SYSTEM$STREAMLIT_NOTEBOOK_WH" TO ROLE PUBLIC;

-- ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_EU';

-- GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE ACCOUNTADMIN;
-- CREATE DATABASE IF NOT EXISTS snowflake_intelligence;
-- CREATE SCHEMA IF NOT EXISTS snowflake_intelligence.agents;
-- GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE ACCOUNTADMIN;
-- GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;
-- GRANT CREATE AGENT ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;

---
## Create Dynamic Table
`HISTORICAL_QUOTES_TYPED` is a copy of the table created by the Openflow Kafka connector, but with correct types for date and currency columns.

In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE HISTORICAL_QUOTES_DEMO;

CREATE OR REPLACE DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED"
    TARGET_LAG = '1 hour'
    WAREHOUSE = 'COMPUTE_WH'
    AS
        SELECT 
            SYMBOL,
            TO_DATE(DATE) AS QUOTE_DATE,
            TO_DOUBLE(LTRIM(CLOSE_LAST, '$')) AS CLOSE_LAST_USD,
            VOLUME,
            TO_DOUBLE(LTRIM(OPEN, '$')) AS OPEN_USD,
            TO_DOUBLE(LTRIM(HIGH, '$')) AS HIGH_USD,
            TO_DOUBLE(LTRIM(LOW, '$')) AS LOW_USD
        FROM "HISTORICAL-STOCK-QUOTES";

-- DESC TABLE "HISTORICAL-STOCK-QUOTES";
-- DESC TABLE "HISTORICAL_QUOTES_TYPED";

In [ ]:
------------------------------
-- Speed things up a bit
------------------------------
ALTER DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED" SET TARGET_LAG = '1 minute';
-- ALTER DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED" SET TARGET_LAG = '1 hour';

In [ ]:
SELECT 
    SYMBOL,
    COUNT(QUOTE_DATE) AS num_quotes,
    MAX(VOLUME),
    MAX(CLOSE_LAST_USD),
    MAX(QUOTE_DATE)
FROM "HISTORICAL_QUOTES_TYPED"
GROUP BY SYMBOL
ORDER BY SYMBOL;

---
## Stock Price Forecast

In [ ]:
-- CREATE OR REPLACE VIEW "TIMESERIES-STOCK-QUOTES" AS 
--     SELECT 
--         SYMBOL,
--         TO_DATE(DATE) AS QUOTE_DATE,
--         TO_DOUBLE(LTRIM(CLOSE_LAST, '$')) AS CLOSE_LAST_USD        
--     FROM "HISTORICAL-STOCK-QUOTES"
--     ORDER BY SYMBOL, QUOTE_DATE;

SELECT * FROM "TIMESERIES-STOCK-QUOTES" LIMIT 10;

In [ ]:
------------------------------
-- Create forecast model
------------------------------
-- CREATE SNOWFLAKE.ML.FORECAST stock_quote_forecast(
--   INPUT_DATA => TABLE("TIMESERIES-STOCK-QUOTES"),
--   SERIES_COLNAME => 'SYMBOL',
--   TIMESTAMP_COLNAME => 'QUOTE_DATE',
--   TARGET_COLNAME => 'CLOSE_LAST_USD'
-- );

In [ ]:
CREATE OR REPLACE TABLE "FORECAST-STOCK-QUOTES" AS
  SELECT * 
  FROM TABLE(stock_quote_forecast!FORECAST(FORECASTING_PERIODS => 183));

In [ ]:
DESC TABLE "FORECAST-STOCK-QUOTES";

-- SELECT * 
-- FROM "FORECAST-STOCK-QUOTES"
-- WHERE SERIES = 'SNOW'
-- ORDER BY "TS" DESC;

---
## Cleanup After Yourself!

In [ ]:
----------------------------------------
-- Don't forget to drop the dynamic table to avoid wasting dollars
----------------------------------------
-- DROP DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED";

In [ ]:
----------------------------------------
-- Truncate the Kafka table
----------------------------------------
-- TRUNCATE TABLE IF EXISTS "HISTORICAL-STOCK-QUOTES";

In [ ]:
----------------------------------------
-- Drop the stock price forecast table and model
----------------------------------------
-- DROP TABLE "FORECAST-STOCK-QUOTES";
-- DROP SNOWFLAKE.ML.FORECAST stock_quote_forecast;